# Load Data

## [Link](https://github.com/fastai/course-v3/blob/master/nbs/swift/00_load_data.ipynb) to the original notebook

In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/saeta/Just", from: "0.7.2")' Just
%install '.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")' NotebookExport

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
	.package(url: "https://github.com/saeta/Just", from: "0.7.2")
		Just
	.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")
		NotebookExport
With SwiftPM flags: []
Working in: /tmp/tmpb_7x4typ/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/3] Merging module jupyterInstalledPackages
Initializing Swift...
Installation complete!


##### Define custom operations

In [2]:
// export
precedencegroup ExponentiationPrecedence {
    associativity: right
    higherThan: MultiplicationPrecedence
}
infix operator ** : ExponentiationPrecedence

precedencegroup CompositionPrecedence { associativity: left }
infix operator >| : CompositionPrecedence

**Infix**: 
Operator is used in between "targets"

https://docs.swift.org/swift-book/LanguageGuide/AdvancedOperators.html


**Precedence**: 
Defines precedence of the operator

https://developer.apple.com/documentation/swift/swift_standard_library/operator_declarations

## Import libraries

- [Just](https://github.com/JustHTTP/Just) does the equivalent of requests in python
- [Path](https://github.com/mxcl/Path.swift) is even better than its python counterpart
- Foundation is the base library from Apple

In [3]:
//export
import Foundation
import Just
import Path

In [4]:
//export
public extension String {
    @discardableResult
    func shell(_ args: String...) -> String
    {
        let (task,pipe) = (Process(),Pipe())
        task.executableURL = URL(fileURLWithPath: self)
        (task.arguments,task.standardOutput) = (args,pipe)
        do    { try task.run() }
        catch { print("Unexpected error: \(error).") }

        let data = pipe.fileHandleForReading.readDataToEndOfFile()
        return String(data: data, encoding: String.Encoding.utf8) ?? ""
    }
}

**@discardableResult**

Ignore return values.

https://www.hackingwithswift.com/example-code/language/how-to-ignore-return-values-using-discardableresult


**Process()**

Process

https://developer.apple.com/documentation/foundation/process


**Pipe()**

Pipe

https://developer.apple.com/documentation/foundation/pipe



In [5]:
print("/bin/ls".shell("-lh"))

total 16K
-rw-r--r-- 1 root root 3.0K Jul 30 20:58 00_load_data.ipynb
-rw-r--r-- 1 1000 1000  225 Jul 30 20:22 README.md
-rw-r--r-- 1 root root 3.4K Jul 30 18:02 hello_world.ipynb
drwxr-xr-x 4 root root 4.0K Jul 30 18:00 swift-install



"[ls](https://www.rapidtables.com/code/linux/ls.html) -lh" -> list long format with readable file size

In [6]:
print("/bin/ls".shell("-l"))

total 16
-rw-r--r-- 1 root root 3071 Jul 30 20:58 00_load_data.ipynb
-rw-r--r-- 1 1000 1000  225 Jul 30 20:22 README.md
-rw-r--r-- 1 root root 3402 Jul 30 18:02 hello_world.ipynb
drwxr-xr-x 4 root root 4096 Jul 30 18:00 swift-install



In [7]:
//export
public func downloadFile(_ url: String, dest: String? = nil, force: Bool = false) {
    let dest_name = dest ?? (Path.cwd/url.split(separator: "/").last!).string
    let url_dest = URL(fileURLWithPath: (dest ?? (Path.cwd/url.split(separator: "/").last!).string))
    if !force && Path(dest_name)!.exists { return }

    print("Downloading \(url)...")

    if let cts = Just.get(url).content {
        do    {try cts.write(to: URL(fileURLWithPath:dest_name))}
        catch {print("Can't write to \(url_dest).\n\(error)")}
    } else {
        print("Can't reach \(url)")
    }
}

In [8]:
downloadFile("https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz")

## Here it comes

In [9]:
//export
import TensorFlow

The following is generic over the element type on the return value. We could define two functions like this:

```swift
func loadMNIST(training: Bool, labels: Bool, path: Path, flat: Bool) -> Tensor<Float> {}
func loadMNIST(training: Bool, labels: Bool, path: Path, flat: Bool) -> Tensor<Int32> {}
```

but that would be boring. So we make loadMNIST take a "type parameter" T which indicates what sort of element type to load a tensor into.


But this doesn't work because S4TF can't just put any type of data inside a Tensor. We have to tell it that this type:

is a type that TF can understand and deal with
is a type that can be applied to the data we read in the byte format
We do this by defining a protocol called ConvertibleFromByte that inherits from TensorFlowScalar. That takes care of the first requirement. The second requirement is dealt with by asking for an init method that takes UInt8

In [10]:
//export
protocol ConvertibleFromByte: TensorFlowScalar {
    init(_ d:UInt8)
}

In [11]:
//export
extension Float : ConvertibleFromByte {}
extension Int32 : ConvertibleFromByte {}

In [12]:
//export
extension Data {
    func asTensor<T:ConvertibleFromByte>() -> Tensor<T> {
        return Tensor(map(T.init))
    }
}

In [13]:
//export
func loadMNIST<T: ConvertibleFromByte>
            (training: Bool, labels: Bool, path: Path, flat: Bool) -> Tensor<T> {
    let split = training ? "train" : "t10k"
    let kind = labels ? "labels" : "images"
    let batch = training ? 60000 : 10000
    let shape: TensorShape = labels ? [batch] : (flat ? [batch, 784] : [batch, 28, 28])
    let dropK = labels ? 8 : 16
    let baseUrl = "https://storage.googleapis.com/cvdf-datasets/mnist/"
    let fname = split + "-" + kind + "-idx\(labels ? 1 : 3)-ubyte"
    let file = path/fname
    if !file.exists {
        downloadFile("\(baseUrl)\(fname).gz", dest:(path/"\(fname).gz").string)
        "/bin/gunzip".shell("-fq", (path/"\(fname).gz").string)
    }
    let data = try! Data(contentsOf: URL(fileURLWithPath: file.string)).dropFirst(dropK)
    if labels { return data.asTensor() }
    else      { return data.asTensor().reshaped(to: shape)}
}

public func loadMNIST(path:Path, flat:Bool = false)
        -> (Tensor<Float>, Tensor<Int32>, Tensor<Float>, Tensor<Int32>) {
    try! path.mkdir(.p)
    return (
        loadMNIST(training: true,  labels: false, path: path, flat: flat) / 255.0,
        loadMNIST(training: true,  labels: true,  path: path, flat: flat),
        loadMNIST(training: false, labels: false, path: path, flat: flat) / 255.0,
        loadMNIST(training: false, labels: true,  path: path, flat: flat)
    )
}

In [14]:
//export
public let mnistPath = Path.home/".fastai"/"data"/"mnist_tst"

In [15]:
let (xTrain, yTrain, xValid, yValid) = loadMNIST(path: mnistPath)
xTrain.shape

▿ [60000, 28, 28]
  ▿ dimensions : 3 elements
    - 0 : 60000
    - 1 : 28
    - 2 : 28


In [16]:
let (xTrain, yTrain, xValid, yValid) = loadMNIST(path: mnistPath, flat: true)
xTrain.shape

▿ [60000, 784]
  ▿ dimensions : 2 elements
    - 0 : 60000
    - 1 : 784


## Timing

In [17]:
//export 
import Dispatch

// ⏰Time how long it takes to run the specified function, optionally taking
// the average across a number of repetitions.
public func time(repeating: Int = 1, _ f: () -> ()) {
    guard repeating > 0 else { return }
    
    // Warmup
    if repeating > 1 { f() }
    
    var times = [Double]()
    for _ in 1...repeating {
        let start = DispatchTime.now()
        f()
        let end = DispatchTime.now()
        let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
        let milliseconds = nanoseconds / 1e6
        times.append(milliseconds)
    }
    print("average: \(times.reduce(0.0, +)/Double(times.count)) ms,   " +
          "min: \(times.reduce(times[0], min)) ms,   " +
          "max: \(times.reduce(times[0], max)) ms")
}

In [18]:
time(repeating: 10) {
    _ = loadMNIST(training: false, labels: false, path: mnistPath, flat: false) as Tensor<Float>
}

average: 308.656734 ms,   min: 305.301622 ms,   max: 324.08724 ms


## Export

In [19]:
// export
public extension String {
    func findFirst(pat: String) -> Range<String.Index>? {
        return range(of: pat, options: .regularExpression)
    }
    func hasMatch(pat: String) -> Bool {
        return findFirst(pat:pat) != nil
    }
}

In [20]:
//export
public func notebookToScript(fname: Path){
    let newname = fname.basename(dropExtension: true)+".swift"
    let url = fname.parent/"FastaiNotebooks/Sources/FastaiNotebooks"/newname
    do {
        let data = try Data(contentsOf: fname.url)
        let jsonData = try JSONSerialization.jsonObject(with: data, options: .allowFragments) as! [String: Any]
        let cells = jsonData["cells"] as! [[String:Any]]
        var module = """
/*
THIS FILE WAS AUTOGENERATED! DO NOT EDIT!
file to edit: \(fname.lastPathComponent)

*/
        
"""
        for cell in cells {
            if let source = cell["source"] as? [String], !source.isEmpty, 
                   source[0].hasMatch(pat: #"^\s*//\s*export\s*$"#) {
                module.append("\n" + source[1...].joined() + "\n")
            }
        }
        try module.write(to: url, encoding: .utf8)
    } catch {
        print("Can't read the content of \(fname)")
    }
}

In [21]:
// export
public func exportNotebooks(_ path: Path) {
    for entry in try! path.ls()
    where entry.kind == Entry.Kind.file && 
          entry.path.basename().hasMatch(pat: #"^\d*_.*ipynb$"#) {
        print("Converting \(entry)")
        notebookToScript(fname: entry.path)
    }
}

In [22]:
notebookToScript(fname: Path.cwd/"00_load_data.ipynb")

Can't read the content of /notebooks/fastai_p2_v3/00_load_data.ipynb


In [23]:
import NotebookExport
let exporter = NotebookExport(Path.cwd/"00_load_data.ipynb")
print(exporter.export(usingPrefix: "FastaiNotebook_"))

success
